In [17]:
import pandas as pd
import numpy as np
import matplotlib as plot
import pickle


In [18]:
dataset=pd.read_csv("CKD.csv")
#dataset

In [19]:
#dataset.columns

In [20]:
dataset=pd.get_dummies(dataset, drop_first=True, dtype=int)
#dataset

In [21]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [22]:
#Split the data
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset[['classification_yes']]

In [23]:
#independent,dependent


In [24]:
#Split the datas for training and test data set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30, random_state=0)
#x_train,x_test

In [25]:
#scale the data to a range using Standard scaler
from sklearn.preprocessing import StandardScaler
SS=StandardScaler()
x_train=SS.fit_transform(x_train)
x_test=SS.transform(x_test)
#x_train,x_test

In [26]:
SSX=pickle.dump(SS,open("SS.pkl",'wb'))

In [27]:
#Model creation using CV search and linear model
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid={'penalty':["l1", "l2", "elasticnet", None],
            'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
            }    
model= GridSearchCV(LogisticRegression(),param_grid,scoring='f1_weighted', refit=True, n_jobs=-1,cv=5)
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/p

In [28]:
#finding the best parameters
print("Best parameters {}".format(model.best_params_))

Best parameters {'penalty': 'l2', 'solver': 'lbfgs'}


In [37]:
#finding the CV results
cv_results=model.cv_results_
table=pd.DataFrame.from_dict(cv_results)
#print(table)

In [30]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(y_test,y_pred)
print(confusion_matrix)

[[45  0]
 [ 1 74]]


In [31]:
#Classification report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print (clf_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        45
           1       1.00      0.99      0.99        75

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [32]:
#fining ROC_AUC_score
from sklearn.metrics import roc_auc_score
roc_auc_score=roc_auc_score(y_test,model.predict_proba(x_test)[:,1])
print(roc_auc_score)

1.0


In [33]:
#Save the model
pickle.dump(model,open("Final_model_Logistic_classification",'wb'))


In [34]:
#load the model
loaded_model=pickle.load(open("Final_model_Logistic_classification",'rb'))


In [35]:
#display the first record with the all the column
first_rec=dataset.iloc[0]
print(first_rec)

age                      2.000000
bp                      76.459948
al                       3.000000
su                       0.000000
bgr                    148.112676
bu                      57.482105
sc                       3.077356
sod                    137.528754
pot                      4.627244
hrmo                    12.518156
pcv                     38.868902
wc                    8408.191126
rc                       4.705597
sg_b                     0.000000
sg_c                     1.000000
sg_d                     0.000000
sg_e                     0.000000
rbc_normal               1.000000
pc_normal                0.000000
pcc_present              0.000000
ba_present               0.000000
htn_yes                  0.000000
dm_yes                   0.000000
cad_yes                  0.000000
appet_yes                1.000000
pe_yes                   1.000000
ane_yes                  0.000000
classification_yes       1.000000
Name: 0, dtype: float64


In [36]:
input=SS.transform([[2,76,3,0,148,57,3,137,4,12,38,8408,4,0,1,0,0,1,0,0,0,0,0,0,1,1,0]])
model.predict(input)



/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([1])